## Competition: Child Mind Institute - Problematic Internet Usage

- ## Prédiction de l'utilisation problématique d'Internet chez les enfants
- ## Utilisation de modèles deep learning pour analyser l'activité physique




In [86]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm


import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
from tensorflow import keras
import time
import os
import numpy as np

from tensorflow.keras import layers
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
import time
import os
import numpy as np
import pandas as pd


In [87]:
# Vérification environnement Kaggle
IN_KAGGLE = 'KAGGLE_KERNEL_RUN_TYPE' in os.environ

# Configuration du chemin des données
if IN_KAGGLE:
    DATA_PATH = '/kaggle/input/child-mind-institute-problematic-internet-use'
else:
    DATA_PATH = '.'
print(f'IN_KAGGLE: {IN_KAGGLE}')

IN_KAGGLE: False


In [88]:
# Configuration de TensorBoard pour Kaggle
def setup_tensorboard():
    """
    Configure TensorBoard en fonction de l'environnement
    """
    if IN_KAGGLE:
        log_dir = '/kaggle/working/logs'
    else:
        log_dir = 'logs'
    
    os.makedirs(log_dir, exist_ok=True)
    return log_dir

# Configuration des seeds pour reproductibilité
def set_seeds(seed=25):
    """
    Configure les seeds pour la reproductibilité
    """
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.keras.utils.set_random_seed(seed)

In [89]:
def load_accelerometer_data(data_path, is_train=True):
    """
    Charge les données d'accéléromètre en ignorant les fichiers système
    """

    
    def process_file(filename, dirname):
        """Traite un fichier parquet individuel"""
        try:
            df = pd.read_parquet(os.path.join(dirname, filename, 'part-0.parquet'))
            
            # Calcul des statistiques par colonnes
            stats = []
            for col in ['X', 'Y', 'Z', 'enmo', 'anglez']:
                desc = df[col].describe()
                stats.extend([desc['mean'], desc['std'], desc['max']])
            
            stats.extend([
                df['non-wear_flag'].mean(),
                df['light'].mean(),
                df['weekday'].mean()
            ])
            
            return stats, filename.split('=')[1]
        except Exception as e:
            print(f"Erreur lors du traitement du fichier {filename}: {str(e)}")
            return None
    
    # chemin du dossier
    dirname = os.path.join(data_path, 'series_train.parquet' if is_train else 'series_test.parquet')
    
    # Liste des IDs (sous-dossiers) en filtrant autres fichiers cachés (pa ex .DS_Store sur Mac OS)
    ids = [d for d in os.listdir(dirname) 
           if not d.startswith('.') and  # Ignore les fichiers cachés
           os.path.isdir(os.path.join(dirname, d)) and  # Vérifie que c'est un dossier
           d.startswith('id=')]  # Vérifie que c'est un dossier d'ID
    
    print(f"Traitement de {len(ids)} fichiers valides...")
    # Traitement parallèle
    valid_results = []
    with ThreadPoolExecutor() as executor:
        futures = list(tqdm(
            executor.map(lambda fname: process_file(fname, dirname), ids),
            total=len(ids)
        ))
        valid_results = [r for r in futures if r is not None]
    
    if not valid_results:
        raise ValueError("Aucune donnée n'a pu être extraite")
    
    # Séparation statistiques et des IDs
    stats, indexes = zip(*valid_results)
    
    # noms de colonnes (accelerometre)
    col_names = []
    for col in ['X', 'Y', 'Z', 'enmo', 'anglez']:
        col_names.extend([f"accel_{col}_mean", f"accel_{col}_std", f"accel_{col}_max"])
    col_names.extend(['accel_nonwear_mean', 'accel_light_mean', 'accel_weekday_mean'])
    
    # DataFrame accel chargé
    accel_features = pd.DataFrame(stats, columns=col_names)
    accel_features['id'] = indexes
    
    return accel_features

def load_data():
    """
    Charge données .csv et .parquet
    """
    print(f"Chargement des données depuis {DATA_PATH}")
    
    # Chargement données tabulaires
    train_df = pd.read_csv(os.path.join(DATA_PATH, 'train.csv'))
    test_df = pd.read_csv(os.path.join(DATA_PATH, 'test.csv'))
    
    print(f"Dimensions des données tabulaires:")
    print(f"Train: {train_df.shape}")
    print(f"Test: {test_df.shape}")
    
    # Chargement données d'accéléromètre
    print("Chargement des données d'accéléromètre...")
    try:
        accel_train = load_accelerometer_data(DATA_PATH, is_train=True)
        accel_test = load_accelerometer_data(DATA_PATH, is_train=False)
        
        print(f"Dimensions des données d'accéléromètre:")
        print(f"Train: {accel_train.shape}")
        print(f"Test: {accel_test.shape}")
        
    except Exception as e:
        print(f"Erreur lors du chargement des données d'accéléromètre: {str(e)}")
        raise
    
    return train_df, test_df, accel_train, accel_test

In [90]:
def create_missing_indicators(df):
    """
    Crée des indicateurs de val manquantes pour chaque colonne
    """
    missing_indicators = pd.DataFrame()
    for col in df.select_dtypes(include=['float64', 'int64']).columns:
        if df[col].isnull().any():
            missing_indicators[f'{col}_missing'] = df[col].isnull().astype(int)
    return missing_indicators


In [91]:
# Non-utilisé pour le moment
def impute_missing_accel_features(features_df, all_ids, accel_features):
    """
    Impute les valeurs manquantes pour les IDs sans données d'accéléromètre
    """
    # Créer un DataFrame avec tous les IDs
    complete_features = pd.DataFrame(index=all_ids)
    
    # Ajouter un flag pour indiquer la présence de données d'accéléromètre
    complete_features['has_accel_data'] = complete_features.index.isin(accel_features.index).astype(int)
    
    # Fusionner avec les features existantes
    complete_features = complete_features.join(accel_features)
    
    # Imputation des valeurs manquantes pour chaque type de feature
    for col in complete_features.columns:
        if col == 'has_accel_data':
            continue
            
        if 'non-wear_flag' in col:
            complete_features[col].fillna(1, inplace=True)
        elif any(x in col for x in ['X_', 'Y_', 'Z_', 'anglez_', 'enmo_', 'light_']):
            complete_features[col].fillna(0, inplace=True)
        else:  # weekday et time_of_day
            complete_features[col].fillna(complete_features[col].mean(), inplace=True)
    
    return complete_features


In [92]:
def process_categorical_seasons(df):
    """
    Convertit les colonnes de saisons en variables dummy
    """
    season_columns = [col for col in df.columns if col.endswith('Season')]
    for col in season_columns:
        if col in df.columns:
            season_dummies = pd.get_dummies(df[col], prefix=col, dummy_na=True)
            df = pd.concat([df, season_dummies], axis=1)
            df = df.drop(col, axis=1)
    return df

def preprocess_tabular_data(train_df, test_df, target_col='sii'):
    """
    Prétraite les données tabulaires
    """
    # 1. Suppression des lignes sans target dans train_df
    train_df = train_df.dropna(subset=[target_col])
    
    # 2. Identification des colonnes communes car on veut matcher les variables dans les 2 datasets
    common_columns = list(set(test_df.columns) & set(train_df.columns))
    train_df = train_df[common_columns + [target_col]]
    
    # 3. Séparation des colonnes numériques et catégorielles car traitement différent pour chacun
    numeric_columns = train_df.select_dtypes(include=['float64', 'int64', 'float32', 'int32']).columns
    numeric_columns = [col for col in numeric_columns if col != target_col]
    
    # 4. Indicateurs de valeurs manquantes
    train_missing = create_missing_indicators(train_df)
    test_missing = create_missing_indicators(test_df)
    
    # 5. Traitement variables catégorielles (saisons)
    train_df = process_categorical_seasons(train_df)
    test_df = process_categorical_seasons(test_df)
    
    # 6. Imputation KNN pour variables numériques
    imputer = KNNImputer(n_neighbors=5)
    train_numeric = train_df[numeric_columns].copy()
    test_numeric = test_df[numeric_columns].copy()
    
    train_numeric_imputed = pd.DataFrame(
        imputer.fit_transform(train_numeric),
        columns=train_numeric.columns,
        index=train_numeric.index
    )
    test_numeric_imputed = pd.DataFrame(
        imputer.transform(test_numeric),
        columns=test_numeric.columns,
        index=test_numeric.index
    )
    
    # 7. Standardisation des variables numériques, on pourrait également utiliser RobustScaler puis MinMaxScaler
    scaler = StandardScaler()
    train_numeric_scaled = pd.DataFrame(
        scaler.fit_transform(train_numeric_imputed),
        columns=train_numeric_imputed.columns,
        index=train_numeric_imputed.index
    )
    test_numeric_scaled = pd.DataFrame(
        scaler.transform(test_numeric_imputed),
        columns=test_numeric_imputed.columns,
        index=test_numeric_imputed.index
    )
    
    # 8. dataset au complet, on concatène les variables numériques, catégorielles et les indicateurs de valeurs manquantes
    train_processed = pd.concat([
        train_numeric_scaled,
        train_df.select_dtypes(include=['object']),
        train_missing
    ], axis=1)
    
    test_processed = pd.concat([
        test_numeric_scaled,
        test_df.select_dtypes(include=['object']),
        test_missing
    ], axis=1)
    
    # 9. Récupération des targets
    y = train_df[target_col].values
    
    return train_processed, test_processed, y, scaler



def extract_accel_features(df):
    """
    Extrait features statistiques des données d'accéléromètre : accel_ moyenne, std, max, min, médiane
    """
    features = df.groupby('id').agg({
        'X': ['mean', 'std', 'max', 'min', 'median'],
        'Y': ['mean', 'std', 'max', 'min', 'median'],
        'Z': ['mean', 'std', 'max', 'min', 'median'],
        'enmo': ['mean', 'std', 'max', 'min', 'median'],
        'anglez': ['mean', 'std', 'max', 'min', 'median'],
        'non-wear_flag': ['mean', 'sum'],
        'light': ['mean', 'std', 'max', 'min'],
        'weekday': ['mean', 'std'],
        'time_of_day': ['mean', 'std']
    })
    
    # Aplatir les noms de colonnes multi-index
    features.columns = [f"accel_{col[0]}_{col[1]}" for col in features.columns]
    return features


In [93]:
def process_categorical_seasons(df):
    """
    Convertit les colonnes de saisons en variables dummy
    """
    season_columns = [col for col in df.columns if col.endswith('Season')]
    for col in season_columns:
        if col in df.columns:
            season_dummies = pd.get_dummies(df[col], prefix=col, dummy_na=True)
            df = pd.concat([df, season_dummies], axis=1)
            df = df.drop(col, axis=1)
    return df


In [94]:
def prepare_data_for_model(train_df, test_df, accel_train, accel_test, test_size=0.2, random_state=42):
    """
    Prépare les données pour l'entraînement du modèle avec les features d'accéléromètre pré-calculées
    """
    # 1. Suppression des lignes sans target
    train_df = train_df.dropna(subset=['sii'])
    
    # 2. Fusion des données tabulaires & d'accéléromètre
    # Les features sont déjà calculées, on fait directement le merge
    train_df = train_df.merge(accel_train, on='id', how='left')
    test_df = test_df.merge(accel_test, on='id', how='left')
    
    # 3. Identification colonnes communes
    common_columns = list(set(test_df.columns) & set(train_df.columns))
    y = train_df['sii'].values
    train_df = train_df[common_columns]
    
    # 4. Indicateurs valeurs manquantes
    train_missing = create_missing_indicators(train_df)
    test_missing = create_missing_indicators(test_df)
    
    # 5. Traitement variables catégorielles (saisons)
    train_df = process_categorical_seasons(train_df)
    test_df = process_categorical_seasons(test_df)
    
    # 6. Identification des colonnes numériques
    numeric_columns = train_df.select_dtypes(include=['float64', 'int64', 'float32', 'int32']).columns
    
    # 7. Imputation KNN pour variables numériques
    imputer = KNNImputer(n_neighbors=5)
    
    train_numeric = pd.DataFrame(
        imputer.fit_transform(train_df[numeric_columns]),
        columns=numeric_columns,
        index=train_df.index
    )
    
    test_numeric = pd.DataFrame(
        imputer.transform(test_df[numeric_columns]),
        columns=numeric_columns,
        index=test_df.index
    )
    
    # 8. Standardisation
    scaler = StandardScaler()
    train_numeric_scaled = pd.DataFrame(
        scaler.fit_transform(train_numeric),
        columns=train_numeric.columns,
        index=train_numeric.index
    )
    
    test_numeric_scaled = pd.DataFrame(
        scaler.transform(test_numeric),
        columns=test_numeric.columns,
        index=test_numeric.index
    )
    
    # 9. Reconstruction des datasets finaux
    train_processed = pd.concat([
        train_numeric_scaled,
        train_df.select_dtypes(include=['object']),
        train_missing
    ], axis=1)
    
    test_processed = pd.concat([
        test_numeric_scaled,
        test_df.select_dtypes(include=['object']),
        test_missing
    ], axis=1)
    
    # 10. Split avec stratification sur la target
    X_train, X_val, y_train, y_val = train_test_split(
        train_processed, y,
        test_size=test_size,
        random_state=random_state,
        stratify=y
    )
    
    return X_train, X_val, test_processed, y_train, y_val, scaler

# Chargement  données

#data_path = '/kaggle/input/child-mind-institute-problematic-internet-use'
#train_df = pd.read_csv(f'{data_path}/train.csv')
#test_df = pd.read_csv(f'{data_path}/test.csv')
#accel_train = pd.read_parquet(f'{data_path}/series_train.parquet')
#accel_test = pd.read_parquet(f'{data_path}/series_test.parquet')

#train_df = pd.read_csv('train.csv')
#test_df = pd.read_csv('test.csv')
#accel_train = pd.read_parquet('series_train.parquet')
#accel_test = pd.read_parquet('series_test.parquet')

# Preprocessing des données
X_train, X_val, X_test, y_train, y_val, scaler = prepare_data_for_model(
    train_df, test_df, accel_train, accel_test
)

In [95]:
# Configuration initiale (inchangée)
IN_KAGGLE = 'KAGGLE_KERNEL_RUN_TYPE' in os.environ
DATA_PATH = '/kaggle/input/child-mind-institute-problematic-internet-use' if IN_KAGGLE else '.'

# Chargement des données avec la nouvelle approche
train_df, test_df, accel_train, accel_test = load_data()

# Le reste de votre pipeline reste inchangé
X_train, X_val, X_test, y_train, y_val, scaler = prepare_data_for_model(
    train_df, test_df, accel_train, accel_test
)

Chargement des données depuis .
Dimensions des données tabulaires:
Train: (3960, 82)
Test: (20, 59)
Chargement des données d'accéléromètre...
Traitement de 996 fichiers valides...


100%|██████████| 996/996 [00:08<00:00, 115.69it/s]


Traitement de 2 fichiers valides...


100%|██████████| 2/2 [00:00<00:00, 40.14it/s]


Dimensions des données d'accéléromètre:
Train: (996, 19)
Test: (2, 19)


In [96]:
accel_train.id.unique().shape

(996,)

In [97]:
print("Nombre total d'IDs dans test_df:", test_df.id.nunique())
print("Nombre d'IDs dans accel_test:", accel_test.id.nunique())
print("IDs dans test_df:", test_df.id.unique())
print("IDs dans accel_test:", accel_test.id.unique())

Nombre total d'IDs dans test_df: 20
Nombre d'IDs dans accel_test: 2
IDs dans test_df: ['00008ff9' '000fd460' '00105258' '00115b9f' '0016bb22' '001f3379'
 '0038ba98' '0068a485' '0069fbed' '0083e397' '0087dd65' '00abe655'
 '00ae59c9' '00af6387' '00bd4359' '00c0cd71' '00d56d4b' '00d9913d'
 '00e6167c' '00ebc35d']
IDs dans accel_test: ['001f3379' '00115b9f']


In [98]:
X_train_head = X_train.head()
X_val_head = X_val.head()
X_test_head = X_test.head()
y_train_head = y_train[:5]
y_val_head = y_val[:5]

In [99]:
X_train.head()
X_val.head()
X_test, y_train, y_val

(    FGC-FGC_GSD  FGC-FGC_SRR  PAQ_C-PAQ_C_Total  FGC-FGC_PU_Zone  BIA-BIA_FMI  \
 0     -1.020359    -1.068121           0.137091        -0.888445    -0.002662   
 1      0.017831     0.662279          -0.396029        -0.888445    -0.296235   
 2     -0.677891     0.316199          -0.672341         1.539911     0.059101   
 3      0.921192    -0.722041          -0.215613        -0.888445     0.181270   
 4      1.158492    -1.275769          -0.854057        -0.402773     1.154649   
 5     -0.246435     0.662279           2.480869        -0.888445     1.653885   
 6     -0.367782     0.662279           1.765708        -0.888445     0.299128   
 7     -1.163278    -3.144601          -2.135171        -0.888445     0.027893   
 8      1.511746    -0.549001          -0.613828        -0.402773     0.850448   
 9      1.465904    -0.860473          -0.532235         0.082898     1.136931   
 10    -0.472949     0.108551           0.585692        -0.402773     0.132090   
 11    -0.205986

In [ ]:
def prepare_data_for_training(X_train, X_val, X_test, y_train, y_val):
    """
    Prépare les données pour l'entraînement 
    """
    # Suppression des colonnes non numériques
    if isinstance(X_train, pd.DataFrame):
        # Identification des colonnes numériques
        numeric_cols = X_train.select_dtypes(include=['float64', 'float32', 'int64', 'int32']).columns
        X_train = X_train[numeric_cols]
        X_val = X_val[numeric_cols]
        if X_test is not None:
            X_test = X_test[numeric_cols]
    
    # Conversion en array numpy si ce n'est pas déjà fait
    if isinstance(X_train, pd.DataFrame):
        X_train = X_train.astype('float32').to_numpy()
    if isinstance(X_val, pd.DataFrame):
        X_val = X_val.astype('float32').to_numpy()
    if X_test is not None and isinstance(X_test, pd.DataFrame):
        X_test = X_test.astype('float32').to_numpy()
        
    # Conversion des labels en int32
    y_train = y_train.astype('int32')
    y_val = y_val.astype('int32')
    
    return X_train, X_val, X_test, y_train, y_val

def create_linear_model(input_shape):
    """
    modèle linéaire
    """
    model = keras.Sequential([
        layers.Input(shape=input_shape),
        layers.Dense(4, activation='softmax', name='output')
    ])
    return model

def create_mlp_model(input_shape, dropout_rate=0.3):
    """
    Crée un MLP
    """
    model = keras.Sequential([
        layers.Input(shape=input_shape),
        layers.Dense(256, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(dropout_rate),
        layers.Dense(128, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(dropout_rate),
        layers.Dense(64, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(dropout_rate),
        layers.Dense(4, activation='softmax', name='output')
    ])
    return model

def create_cnn_model(input_shape, dropout_rate=0.3):
    """
    Crée un CNN 1D notamment pour données tabulaires
    """
    model = keras.Sequential([
        layers.Input(shape=input_shape),
        layers.Reshape((-1, 1)),
        layers.Conv1D(32, 3, activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling1D(2),
        layers.Dropout(dropout_rate),
        layers.Conv1D(64, 3, activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling1D(2),
        layers.Dropout(dropout_rate),
        layers.Conv1D(128, 3, activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.GlobalAveragePooling1D(),
        layers.Dropout(dropout_rate),
        layers.Dense(128, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(dropout_rate),
        layers.Dense(4, activation='softmax', name='output')
    ])
    return model

def train_model(model, X_train, y_train, X_val, y_val, model_name, batch_size=32, epochs=100):
    """
    Fonction d'entraînement modèle (avec TensorBoard et EarlyStopping)
    """
    # Création du dossier pour les logs TensorBoard
    log_dir = os.path.join("logs", model_name + "_" + time.strftime("%Y%m%d-%H%M%S"))
    os.makedirs(log_dir, exist_ok=True)
    
    tensorboard_callback = TensorBoard(
        log_dir=log_dir,
        histogram_freq=1,
        write_graph=True,
        write_images=True
    )
    
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True
    )
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    # résumé du modèle
    model.summary()
    
    # Entraînement
    history = model.fit(
        X_train, y_train,
        batch_size=batch_size,
        epochs=epochs,
        validation_data=(X_val, y_val),
        callbacks=[tensorboard_callback, early_stopping],
        verbose=1
    )
    
    return history

def evaluate_model(model, X_val, y_val):
    """
    Évaluation et métriques modèles
    """
    val_loss, val_acc = model.evaluate(X_val, y_val, verbose=0)
    y_pred = model.predict(X_val)
    y_pred_classes = np.argmax(y_pred, axis=1)
    
    return {
        'loss': val_loss,
        'accuracy': val_acc,
        'predictions': y_pred,
        'pred_classes': y_pred_classes
    }

def train_all_models(X_train, X_val, y_train, y_val, input_shape):
    """
    Entraine et retourne leurs performances
    """
    X_train, X_val, _, y_train, y_val = prepare_data_for_training(
        X_train, X_val, None, y_train, y_val
    )
    
    input_shape = (X_train.shape[1],)
    
    models = {
        'linear': create_linear_model(input_shape),
        'mlp': create_mlp_model(input_shape),
        'cnn': create_cnn_model(input_shape)
    }
    
    histories = {}
    evaluations = {}
    
    for name, model in models.items():
        print(f"\nEntraînement du modèle {name}")
        history = train_model(model, X_train, y_train, X_val, y_val, name)
        histories[name] = history
        
        # Évaluation du modèle
        eval_results = evaluate_model(model, X_val, y_val)
        evaluations[name] = eval_results
        
        print(f"{name} - Validation Loss: {eval_results['loss']:.4f}, "
              f"Validation Accuracy: {eval_results['accuracy']:.4f}")
    
    # Détermination du meilleur modèle
    best_model_name = min(evaluations.items(), key=lambda x: x[1]['loss'])[0]
    print(f"\nMeilleur modèle (basé sur la validation loss): {best_model_name}")
    
    return models, histories, best_model_name, evaluations

def generate_submission(best_model, X_test, test_df, output_file='submission.csv'):
    """
    Génère le fichier de soumission avec les prédictions
    """
    # Garde copie de l'ID avant préparation
    test_ids = test_df['id'].copy()
    
    # Sélection uniquement des colonnes numériques
    numeric_cols = X_test.select_dtypes(include=['float64', 'float32', 'int64', 'int32']).columns
    X_test_numeric = X_test[numeric_cols].copy()
    
    # Conversion array numpy
    X_test_prepared = X_test_numeric.astype('float32').to_numpy()
    
    # Génération prédictions
    predictions = best_model.predict(X_test_prepared, verbose=0)
    predictions_classes = np.argmax(predictions, axis=1)
    
    # Création DataFrame de soumission
    submission = pd.DataFrame({
        'id': test_ids,
        'sii': predictions_classes
    })

    if IN_KAGGLE:
        output_file = f'/kaggle/working/{output_file}'
    
    submission.to_csv(output_file, index=False)
    print(f"Fichier de soumission généré : {output_file}")
    
    return submission

In [101]:
input_shape = (X_train.shape[1],)

# Entraînement de tous les modèles
models, histories, best_model_name, evaluations = train_all_models(
    X_train, X_val, y_train, y_val, input_shape
)



Entraînement du modèle linear
Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 output (Dense)              (None, 4)                 516       
                                                                 
Total params: 516 (2.02 KB)
Trainable params: 516 (2.02 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/100
69/69 [==============================] - 0s 1ms/step - loss: 1.3963 - accuracy: 0.4292 - val_loss: 1.1922 - val_accuracy: 0.5255
Epoch 2/100
69/69 [==============================] - 0s 670us/step - loss: 1.1635 - accuracy: 0.5210 - val_loss: 1.0848 - val_accuracy: 0.5347
Epoch 3/100
69/69 [==============================] - 0s 664us/step - loss: 1.0713 - accuracy: 0.5430 - val_loss: 1.0247 - val_accuracy: 0.5529
Epoch 4/100
69/69 [==============================] - 0s 703us/step - loss: 1.0114 - accuracy:

In [102]:

# Pour visualiser dans TensorBoard
# %load_ext tensorboard
# %tensorboard --logdir logs/


In [103]:
test_df_head = test_df.head()

In [104]:
# Génération du fichier de soumission avec le meilleur modèle
best_model = models[best_model_name]
submission = generate_submission(best_model, X_test, test_df)

# Vérification du contenu du fichier de soumission
print("\nAperçu du fichier de soumission :")
print(submission.head())
print("\nDistribution des prédictions :")
print(submission['sii'].value_counts())

Fichier de soumission généré : submission.csv

Aperçu du fichier de soumission :
         id  sii
0  00008ff9    0
1  000fd460    0
2  00105258    0
3  00115b9f    0
4  0016bb22    0

Distribution des prédictions :
0    20
Name: sii, dtype: int64
